In [1]:
import sys
import os
from pathlib import Path
# This appends the directory one level up (the root of your project) to the sys.path.
# Modify the path depending on the location of modules you want to import.
sys.path.append(os.path.abspath('../'))

In [2]:
from config.config_managers import DashboardConfigManager
from dataManager import DataManager

2024-12-10 09:50:07 - INFO - PyTorch version 2.2.2 available.


In [3]:
CONFIG_PATH = Path("/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/dashboard-config.yaml")
config_manager = DashboardConfigManager(CONFIG_PATH)
dev_config = config_manager.development_config    

In [4]:

from dash import Dash, dcc, html, Output, Input, State
app = Dash(__name__, suppress_callback_exceptions=True)

app_config = config_manager.app_config
server = app.server  # Flask server instance for caching
variants_data = None

data_manager = DataManager(config_manager, server)

In [5]:
all_data = data_manager.load_data()

2024-12-10 09:50:08 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/My Drive
2024-12-10 09:50:08 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-12-10 09:50:08 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02


  0%|          | 0/14 [00:00<?, ?it/s]

2024-12-10 09:50:11 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/My Drive
2024-12-10 09:50:11 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-12-10 09:50:11 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/conll2003_bert


  0%|          | 0/14 [00:00<?, ?it/s]

In [10]:
all_data.keys()

dict_keys(['ANERCorp_CamelLab_arabertv02', 'conll2003_bert'])

In [ ]:
anercor

In [ ]:
tr_df = all_data['conll2003_bert'].train_data.copy()
tr_df = all_data['conll2003_bert'].train_data.copy()
tr_df[tr_df['Labels']!=-100]

,Sentence Ids,Token Positions,X,Y,Labels,Losses,Token Ids,Global Id,True Labels
1,0,1,-8.761537,5.038084,3,0.000471,7270,7270_0_1_3,B-ORG
2,0,2,0.028062,-0.254672,0,0.000020,22961,22961_0_2_0,O
3,0,3,5.212746,13.185966,7,0.000606,1528,1528_0_3_7,B-MISC
4,0,4,0.120432,-0.278172,0,0.000020,1840,1840_0_4_0,O
5,0,5,0.050218,-0.425187,0,0.000019,1106,1106_0_5_0,O
...,...,...,...,...,...,...,...,...,...
300669,14039,2,-1.354588,6.421616,0,0.000056,1210,1210_14039_2_0,O
300672,14040,1,-9.390898,5.436679,3,0.000231,17057,17057_14040_1_3,B-ORG
300673,14040,2,6.466621,-7.676339,0,0.000021,122,122_14040_2_0,O
300674,14040,3,-9.392007,5.417524,3,0.000269,4617,4617_14040_3_3,B-ORG


In [6]:
len(all_data['ANERCorp_CamelLab_arabertv02'].analysis_data.columns)

61

In [35]:
all_data['conll2003_bert'].analysis_data.columns[~(all_data['conll2003_bert'].analysis_data.columns == all_data['ANERCorp_CamelLab_arabertv02'].analysis_data.columns)]

Index([], dtype='object')

In [36]:
all_data['conll2003_bert'].analysis_data['B-PER Confidence']

0        0.0004
1        0.0001
2        0.0005
3        0.0003
4        0.0004
          ...  
70362    0.0000
70363    0.0000
70364    0.9995
70365    0.0000
70366    0.0003
Name: B-PER Confidence, Length: 70367, dtype: float64

In [6]:
df = all_data['ANERCorp_CamelLab_arabertv02'].analysis_data.copy()
df = df[df['Labels'] != -100].copy()

In [7]:
df.columns

Index(['Sentence Ids', 'Token Positions', 'Words', 'Tokens', 'Word Pieces',
       'Core Tokens', 'True Labels', 'Token Selector Id', 'Pred Labels',
       'Agreements', 'X', 'Y', 'Labels', 'Losses', 'Token Ids', 'Global Id',
       'True Silhouette Score', 'Pred Silhouette Score', 'K=3',
       'Boundary Clusters', 'K=4', 'Entity Clusters', 'K=9', 'Token Clusters',
       'Consistency Count', 'Inconsistency Count', 'Total Train Occurrences',
       'Local Token Entropy', 'Token Max Entropy', 'Dataset Token Entropy',
       'Local Word Entropy', 'Word Max Entropy', 'Dataset Word Entropy',
       'Tokenization Rate', 'Error Type', 'O Confidence', 'B-PERS Confidence',
       'I-PERS Confidence', 'B-ORG Confidence', 'I-ORG Confidence',
       'B-LOC Confidence', 'I-LOC Confidence', 'B-MISC Confidence',
       'I-MISC Confidence', 'Prediction Entropy', 'Prediction Max Entropy',
       'Token Confidence', 'Variability', 'Pre X', 'Pre Y',
       'Strict True Entities', 'Strict Pred Entities'

In [68]:
df[(df['Strict True Entities'] == 'O') & (df['True Labels'] != 'O') ][['True Entities', 'True Labels', 'True Aligned Scheme']].value_counts()

True Entities  True Labels  True Aligned Scheme
PERS           I-PERS       False                  57
MISC           I-MISC       False                  10
ORG            I-ORG        False                   9
LOC            I-LOC        False                   8
Name: count, dtype: int64

In [60]:
df[(df['Strict Pred Entities'] == 'O')]['Pred Labels'].value_counts()

Pred Labels
O         21783
I-ORG        18
I-MISC       14
I-PERS       10
I-LOC         4
Name: count, dtype: int64

In [61]:
df[(df['Pred Entities'] == 'O')]['Pred Labels'].value_counts()

Pred Labels
O    21783
Name: count, dtype: int64

In [63]:
df[(df['Strict True Entities'] == 'O')]['True Labels'].value_counts()

True Labels
O         21616
I-PERS       57
I-MISC       10
I-ORG         9
I-LOC         8
Name: count, dtype: int64

In [52]:
df['True Labels']

1         B-LOC
2         B-LOC
3             O
4        B-PERS
5        I-PERS
          ...  
29704         O
29705    B-PERS
29707    I-PERS
29708    I-PERS
29709         O
Name: True Labels, Length: 24991, dtype: object

In [34]:
df.columns[(df == -1).any(axis=0)]

Index(['Local Token Entropy', 'Token Max Entropy', 'Dataset Token Entropy',
       'Local Word Entropy', 'Word Max Entropy', 'Dataset Word Entropy',
       'Normalized Token Entropy', 'Normalized Word Entropy'],
      dtype='object')

In [29]:
f = df[df['Variability'] != -1]

In [24]:
f['Normalized Token Entropy'].describe()

count    22340.000000
mean         0.093887
std          0.237697
min          0.000000
25%          0.000000
50%          0.000000
75%          0.009250
max          1.000000
Name: Normalized Token Entropy, dtype: float64

In [27]:
filtered_df = df[~(df == -1).any(axis=1)]

- 
- multi token entity by extract entities and scan the length of the indeces and get the longest one
- pay attention to rare entities because we have opposite behaviour to that, OOV they tend to be predicted fine

In [28]:
filtered_df

,Sentence Ids,Token Positions,Words,Tokens,Word Pieces,Core Tokens,True Labels,Token Selector Id,Pred Labels,Agreements,...,Strict Pred Entities,True Entities,Pred Entities,True Aligned Scheme,Pred Aligned Scheme,Consistency Ratio,Inconsistency Ratio,Normalized Token Entropy,Normalized Word Entropy,Normalized Prediction Entropy
2,0,2,المفرق,المفرق,المفرق,المفرق,B-LOC,المفرق@#2@#0,B-LOC,True,...,LOC,LOC,LOC,True,True,0.000000,1.000000,0.000000,0.000000,0.056689
3,0,3,-,-,-,-,O,-@#3@#0,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.002177
5,0,5,الطراونة,الطر,"الطر, ##او, ##نة",الطر,I-PERS,الطر@#5@#0,I-PERS,True,...,PERS,PERS,PERS,True,True,0.142857,0.857143,0.591700,0.591700,0.073851
8,0,8,-,-,-,-,O,-@#8@#0,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.002398
9,0,9,أمر,أمر,أمر,أمر,O,أمر@#9@#0,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.000158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29703,960,7,الحياة,الحياة,الحياة,الحياة,O,الحياة@#7@#960,O,True,...,O,O,O,True,True,0.968750,0.031250,0.200600,0.200600,0.000379
29704,960,8,الشخصية,الشخصية,الشخصية,الشخصية,O,الشخصية@#8@#960,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.000284
29707,960,11,بن,بن,بن,بن,I-PERS,بن@#11@#960,I-PERS,True,...,PERS,PERS,PERS,True,True,0.903509,0.096491,0.250269,0.250269,0.020158
29708,960,12,طلال,طلال,طلال,طلال,I-PERS,طلال@#12@#960,I-PERS,True,...,PERS,PERS,PERS,True,True,0.500000,0.500000,1.000000,1.000000,0.020348


In [ ]:
all_data['ANERCorp_CamelLab_arabertv02'].entity_non_strict_report

{'confusion_matrix': {'LOC': {'TP': 627, 'FP': 76, 'FN': 49},
  'ORG': {'TP': 338, 'FP': 105, 'FN': 121},
  'MISC': {'TP': 151, 'FP': 54, 'FN': 92},
  'PERS': {'TP': 751, 'FP': 99, 'FN': 154}},
 'false_negatives': {'LOC': {'O': 29,
   'Boundary': 10,
   'PERS': 1,
   'MISC': 3,
   'ORG': 3,
   'Entity and Boundary': 3},
  'PERS': {'O': 50,
   'MISC': 5,
   'Boundary': 75,
   'ORG': 13,
   'LOC': 6,
   'Entity and Boundary': 5},
  'ORG': {'O': 48,
   'Entity and Boundary': 10,
   'Boundary': 26,
   'MISC': 6,
   'PERS': 11,
   'LOC': 20},
  'MISC': {'O': 46,
   'Entity and Boundary': 14,
   'LOC': 6,
   'Boundary': 17,
   'ORG': 9}},
 'false_positives': {'LOC': {'O': 15,
   'Boundary': 14,
   'Entity and Boundary': 15,
   'MISC': 6,
   'PERS': 6,
   'ORG': 20},
  'MISC': {'PERS': 5,
   'O': 18,
   'ORG': 6,
   'LOC': 3,
   'Boundary': 20,
   'Entity and Boundary': 2},
  'ORG': {'Boundary': 31,
   'Entity and Boundary': 5,
   'O': 44,
   'MISC': 9,
   'PERS': 13,
   'LOC': 3},
  'PERS': 

In [1]:
analysis_data = data_manager.variants_data['ANERCorp_CamelLab_arabertv02'].analysis_data
errors = analysis_data[analysis_data['Error Type'] != 'No Errors']

NameError: name 'data_manager' is not defined

In [2]:
import pandas as pd
from collections import defaultdict, Counter
from seqeval.scheme import auto_detect, Entities
from seqeval.metrics.sequence_labeling import get_entities

class EntityAnnotator:
    def __init__(self, y_true, y_pred):
        """Initialize the annotator with true and predicted labels."""
        self.y_true = y_true
        self.y_pred = y_pred
        self.scheme = auto_detect(self.y_true, False)
    
    def extract_entities(self):
        """Extract entities based on the scheme detected."""
        self.entities_strict_true = Entities(self.y_true, self.scheme, False)
        self.entities_strict_pred = Entities(self.y_pred, self.scheme, False)
        self.entities_true = get_entities(self.y_true)
        self.entities_pred = get_entities(self.y_pred)

    def process_strict_entities(self, y_true, entities_true, sen_id):
        """Process entities strictly, labeling full spans in a sentence."""
        max_len = len(y_true[sen_id])
        results = ['O'] * max_len
        for idx in range(max_len):
            for entity in entities_true[sen_id]:
                _, t, s, e = entity.to_tuple()
                if s == idx and (e-s) > 0:
                    for i in range(e-s):
                        results[s + i] = t
                elif (e-s) == 0:
                    results[s] = t
        return results
    
    def process_non_strict_entities(self, y_true, sen_id):
        """Process entities non-strictly, marking only the start and end of each entity."""
        max_len = len(y_true[sen_id])
        results = ['O'] * max_len
        for entity in get_entities(y_true[sen_id]):
            t, s, e = entity
            if s == e:
                # If start and end are the same, only mark the start
                results[s] = t
            else:
                # Mark all indices from start to end inclusive
                for i in range(s, e + 1):
                    results[i] = t
        return results

    def process_sentences(self, analysis_data, y_data, entities, label_column, strict=False):
        """Annotate sentences with entity information, either strictly or non-strictly."""
        entity_annotations = []
        for sentence_id, sentence_df in analysis_data.groupby('Sentence Ids'):
            if strict: 
                results = self.process_strict_entities(y_data, entities, sentence_id)
            else:
                results = self.process_non_strict_entities(y_data, sentence_id)
            original_series = sentence_df[label_column]
            is_metadata = original_series.apply(lambda x: x not in ['[CLS]', '[SEP]', 'IGNORED'])
            new_series = original_series.copy()
            new_series.loc[is_metadata] = results
            entity_annotations.append(new_series)
        return pd.concat(entity_annotations)

    def annotate_entity_info(self, analysis_data):
        """Add annotated entity information to the analysis data for both true and predicted labels."""
        self.extract_entities()  # Ensure entities are extracted before processing
        analysis_data['Strict True Entities'] = self.process_sentences(analysis_data, self.y_true, self.entities_strict_true.entities, 'True Labels', True)
        analysis_data['Strict Pred Entities'] = self.process_sentences(analysis_data, self.y_pred, self.entities_strict_pred.entities, 'Pred Labels', True)
        analysis_data['True Entities'] = self.process_sentences(analysis_data, self.y_true, self.entities_true, 'True Labels')
        analysis_data['Pred Entities'] = self.process_sentences(analysis_data, self.y_pred, self.entities_pred, 'Pred Labels')
        
        return analysis_data


In [3]:
import json
from collections import defaultdict, Counter
from seqeval.scheme import auto_detect
from seqeval.metrics.sequence_labeling import get_entities
from seqeval.scheme import Entities

file_name = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02/fine_tuning/evaluation_metrics.json'
with open(file_name, 'r') as file:
    entity_outputs = json.load(file)  # Use json.load() to read file, not json.loads()

def extract_entities(y_data, scheme):
        # Replace with the Entities() logic if provided
        return Entities(y_data, scheme, False)
core_data = analysis_data[analysis_data['Labels']!= -100].copy()


# y_true = entity_outputs['entity_outputs']['y_true']
y_true = core_data.groupby('Sentence Ids')['True Labels'].apply(list).tolist()
# y_pred = entity_outputs['entity_outputs']['y_pred']
y_pred = core_data.groupby('Sentence Ids')['Pred Labels'].apply(list).tolist()


NameError: name 'analysis_data' is not defined

In [4]:
annotator = EntityAnnotator(y_true, y_pred)
updated_analysis_data = annotator.annotate_entity_info(analysis_data)


NameError: name 'y_true' is not defined

In [5]:
updated_analysis_data[['True Entities', 'TR Entity']]

NameError: name 'updated_analysis_data' is not defined

In [6]:
(updated_analysis_data['True Entities'] != updated_analysis_data['TR Entity']).value_counts()

NameError: name 'updated_analysis_data' is not defined

In [1]:
updated_analysis_data[updated_analysis_data['True Entities'] != updated_analysis_data['Strict True Entities']][['Sentence Ids', 'True Entities', 'TR Entity']]

NameError: name 'updated_analysis_data' is not defined

In [2]:
sen_id = 15
get_entities(y_true[sen_id])

NameError: name 'get_entities' is not defined

In [3]:
ts = updated_analysis_data[updated_analysis_data['Sentence Ids'] == sen_id]

ts[~ts['True Labels'].isin(['[CLS]', '[SEP]', 'IGNORED'])][['Sentence Ids', 'True Entities', 'Strict True Entities', 'Pred Entities', 'Strict Pred Entities','Pred Labels', 'True Labels']].reset_index()

NameError: name 'updated_analysis_data' is not defined

In [ ]:
scheme = auto_detect(y_true, False)
entities_true = extract_entities(y_true)
entities_pred = extract_entities(y_pred)
true_entities = get_entities(y_true)

In [ ]:
def process_strict_entities(y_true, entities_true, sen_id):
    """
    Process entities for a given sentence ID and return a list of results where entities are labeled,
    and non-entity indices are marked as 'O'.

    Args:
    y_true (dict): Dictionary where keys are sentence IDs and values are lists of true labels.
    entities_true (dict): Dictionary where keys are sentence IDs and values are lists of entity objects with a to_tuple method.
    sen_id (int): Sentence ID to process.

    Returns:
    list: A list of labels with entities labeled accordingly and other indices as 'O'.
    """
    # Determine the length needed for the results list
    max_len = len(y_true[sen_id])

    # Initialize results with 'O' for all expected indices
    results = ['O'] * max_len

    # Iterate over each index in the range of y_true for the given sentence ID
    for idx in range(max_len):
        # Check for entities at this index
        for entity in entities_true[sen_id]:
            _, t, s, e = entity.to_tuple()
            # Check if the entity starts at this index and has length
            if s == idx and (e-s) > 0:
                for i in range(e-s):
                    results[s + i] = t  # Replace 'some_label' with your intended label
            elif (e-s) == 0:
                results[s] = t  # Set your intended label for zero-length entities
            # If none of the conditions are met, it will retain 'O'

    return results


In [ ]:
analysis_data.columns

In [ ]:
entity_annotations = []
for sentence_id, sentence_df in analysis_data.groupby('Sentence Ids'):
    results = process_strict_entities(y_true, entities_true.entities, sentence_id)
    original_series = sentence_df['True Labels']
    is_metadata = original_series.apply(lambda x: x not in ['[CLS]', '[SEP]', 'IGNORED'])
    new_series = original_series.copy()
    new_series.loc[is_metadata] = results
    entity_annotations.append(new_series)


In [ ]:
pd.concat(entity_annotations)

In [ ]:
sen_id = 18
data = analysis_data[~analysis_data['TR Entity'].isin(['[CLS]', '[SEP]', 'IGNORED'])]
sen_data = data[data['Sentence Ids'] == sen_id].reset_index()
sen_data[['Sentence Ids', 'True Labels', 'TR Entity']]

In [ ]:
lresults

In [ ]:
# Determine the length needed for the results list
max_len = len(y_true[sen_id])

# Initialize results with 'O' for all expected indices
results = ['O'] * max_len

# Iterate over each index in the range of y_true[694]
for idx in range(max_len):
    # Check for entities at this index
    for entity in entities_true.entities[sen_id]:
        sen_id, t, s, e = entity.to_tuple()
        # Check if the entity starts at this index and has length
        if s == idx and (e-s) > 0:
            for i in range(e-s):
                results[s + i] = t  # Replace 'some_label' with your intended label
        elif (e-s) == 0:
            results[s] = t  # Again, set your intended label for zero-length entities
        # If none of the conditions are met, it will retain 'O'


In [ ]:
# Determine the length needed for the results list
max_len = len(y_true[sen_id])

# Initialize results with 'O' for all expected indices
lresults = ['O'] * max_len

# Iterate over each index in the range of y_true[694]
for idx in range(max_len):
    # Check for entities at this index
    for entity in get_entities(y_true[sen_id]):
        t, s, e = entity
        # Check if the entity starts at this index and has length
        
        if s == e:
            # If start and end are the same, only mark the start
            lresults[s] = t
        else:
            # Mark all indices from start to end inclusive
            for i in range(s, e + 1):
                lresults[i] = t
        


In [ ]:
lresults

In [ ]:
original_series = analysis_data[analysis_data["Sentence Ids"] == sen_id]['True Labels']
is_metadata = original_series.apply(lambda x: x not in ['[CLS]', '[SEP]', 'IGNORED'])


In [ ]:
new_series = original_series.copy()
non_metadata_indices = original_series.index[is_metadata]
new_series.loc[non_metadata_indices] = results


In [ ]:
original_series

In [ ]:
sen_data['TR Entity']

In [ ]:
new_series

In [ ]:
get_entities(y_true[sen_id])

In [ ]:
entities_true.entities[sen_id]


In [ ]:
for i, e in enumerate(lresults
):
    print(i, e)

In [ ]:
# Determine the length needed for the results list
max_len = len(y_true[0])

# Initialize results with 'O' for all expected indices
results = ['O'] * max_len

# Iterate over each index in the range of y_true[694]
for idx in range(max_len):
    # Check for entities at this index
    for entity in entities_true.entities[0]:
        sen_id, t, s, e = entity.to_tuple()
        # Check if the entity starts at this index and has length
        if s == idx and (e-s) > 0:
            for i in range(e-s):
                results[s + i] = t  # Replace 'some_label' with your intended label
        elif (e-s) == 0:
            results[s] = t  # Again, set your intended label for zero-length entities
        # If none of the conditions are met, it will retain 'O'


In [ ]:
results

In [ ]:
errors[errors['Error Type'] == 'Type'][['True Labels', 'Pred Labels', 'Error Type']].sample(60)

In [ ]:
errors[['True Labels', 'Pred Labels', 'Error Type']].sample(60)

In [ ]:
def annotate_error(true_label, pred_label):
        # If both are the same, it's correct (no error)
        if true_label == pred_label:
            return "No Errors"
        
        # Handle cases where one or both labels are 'O'
        if true_label == 'O' and pred_label != 'O':
            return "Chunk"  # False entity predicted
        if true_label != 'O' and pred_label == 'O':
            return "Exclusion"  # Missed entity and chunk boundary
        
        # Extract entity types without position tags (like "B-", "I-")
        true_entity = true_label.split("-")[-1] if "-" in true_label else true_label
        pred_entity = pred_label.split("-")[-1] if "-" in pred_label else pred_label

        # If entity types are different (e.g., LOC vs. PER)
        if true_entity != pred_entity:
            # If both entity type and position (B- vs I-) are wrong
            return "Type and Chunk" if true_label[0] != pred_label[0] else "Type"

        # If entity types are the same but position tags (B- vs I-) are wrong
        return "Chunk"


In [ ]:
import json

file_name = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02/fine_tuning/evaluation_metrics.json'
with open(file_name, 'r') as file:
    entity_outputs = json.load(file)  # Use json.load() to read file, not json.loads()


In [ ]:
y_true = entity_outputs['entity_outputs']['y_true']
y_pred = entity_outputs['entity_outputs']['y_pred']

In [ ]:
from seqeval.metrics import f1_score, classification_report, f1_score
from seqeval.scheme import IOB2

print(classification_report(y_true, y_pred, mode='strict', digits=4))

f1_score(y_true, y_pred, mode='strict', scheme=IOB2,  average='micro')


In [ ]:
from seqeval.metrics import f1_score, classification_report

print(classification_report(y_true, y_pred, mode=None, digits=4))


In [ ]:
from seqeval.metrics import f1_score

# Ground truth (true labels)
y_true = [
    ['O', 'B-PER', 'I-PER', 'O', 'B-LOC', 'O'],
    ['O', 'B-ORG', 'I-ORG', 'O']
]

# Predicted labels (with minor errors)
y_pred = [
    ['O', 'B-PER', 'O', 'O', 'B-LOC', 'O'],  # Misses I-PER
    ['O', 'B-ORG', 'O', 'O']                # Misses I-ORG
]

# Default mode
f1_default = f1_score(y_true, y_pred, average='micro', mode=None)

# Strict mode
f1_strict = f1_score(y_true, y_pred, average='micro', mode='strict')

print("Default Mode F1 Score:", f1_default)
print("Strict Mode F1 Score:", f1_strict)


In [ ]:
from seqeval.metrics import f1_score, classification_report
from seqeval.scheme import IOB1
# Define data
y_true = [['B-PER', 'I-PER', 'O', 'B-ORG', 'I-ORG', 'O']]
y_pred = [['B-PER', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'O']]

# Strict mode, no scheme
# f1_no_scheme_strict = f1_score(y_true, y_pred, average='micro', mode='strict', scheme=None)
f1_no_scheme_strict = classification_report(y_true, y_pred, mode='strict', zero_division='Warn')
no_strict = classification_report(y_true, y_pred, mode=None,)

print("Strict Mode with No Scheme F1 Score:", f1_no_scheme_strict)

print("No Strict Mode with F1 Score:", no_strict)




In [ ]:

entity_y_true = get_entities(y_true)
entity_y_pred = get_entities(y_pred)

# Example usage
# conf_matrix = calculate_confusion_matrix([e.to_tuple()[1:] for sen in entities_true.entities for e in sen], [e.to_tuple()[1:] for sen in entities_pred.entities for e in sen])



conf_matrix = calculate_confusion_matrix(entity_y_true, entity_y_pred)
print(conf_matrix)
fn_errors = compute_false_negatives(entity_y_true, entity_y_pred)
fp_errors = compute_false_positives(entity_y_true, entity_y_pred)

print("False Negatives:", dict(fn_errors))
print("False Positives:", dict(fp_errors))
1

In [ ]:
entity_y_true

In [ ]:
entity_y_pred

In [ ]:


entities_true = Entities(y_true, scheme, False)
entities_pred = Entities(y_pred, scheme, False)
true_entity_type = flatten_strict_entities(entities_true)
pred_entity_type = flatten_strict_entities(entities_pred)
# Example usage
# conf_matrix = calculate_confusion_matrix([e.to_tuple()[1:] for sen in entities_true.entities for e in sen], [e.to_tuple()[1:] for sen in entities_pred.entities for e in sen])



conf_matrix = calculate_confusion_matrix(true_entity_type, pred_entity_type)
print(conf_matrix)
fn_errors = compute_false_negatives(true_entity_type, pred_entity_type)
fp_errors = compute_false_positives(true_entity_type, pred_entity_type)

print("False Negatives:", dict(fn_errors))
print("False Positives:", dict(fp_errors))
1

In [ ]:
entities_true.entities

In [ ]:
entities_pred.entities

In [ ]:
print(no_strict)

In [ ]:
from seqeval.scheme import IOB2, Tokens, auto_detect


In [ ]:
scheme = auto_detect(y_true, False)

In [ ]:
scheme

In [ ]:
tokens = Tokens(['B-PER', 'I-PER', 'O', 'B-LOC'], scheme)

In [ ]:
# outputs = {
#     'y_true': [['O', 'O', 'B-MISC', 'I-MISC', 'B-MISC', 'O', 'O'], ['B-PER', 'I-PER', 'O']],
# 	'y_pred': [['O', 'O', 'B-MISC', 'I-LOC', 'B-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]

# }


In [ ]:
from seqeval.metrics.sequence_labeling import get_entities
# Example usage
entity_y_true = get_entities(entity_outputs['entity_outputs']['y_true'])
entity_y_pred = get_entities(entity_outputs['entity_outputs']['y_pred'])

from seqeval.scheme import Entities
entities_true = Entities(entity_outputs['entity_outputs']['y_true'], scheme, False)
entities_pred = Entities(entity_outputs['entity_outputs']['y_pred'], scheme, False)



In [ ]:
from collections import defaultdict, Counter

def flatten_strict_entities(entities):
    return [e.to_tuple()[1:] for sen in entities.entities for e in sen]

def calculate_confusion_matrix(y_true, y_pred):
    # Initialize confusion matrix data structure
    types = set([ent[0] for ent in y_true]).union([ent[0] for ent in y_pred])
    confusion_matrix = {typ: {'TP': 0, 'FP': 0, 'FN': 0} for typ in types}


    # Track matched predictions to avoid counting them more than once
    matched_pred_indices = set()

    # Check each true entity against predicted entities
    for true_ent in y_true:
        true_type, true_start, true_end = true_ent
        match_found = False

        for idx, pred_ent in enumerate(y_pred):
            pred_type, pred_start, pred_end = pred_ent

            if idx not in matched_pred_indices and true_type == pred_type and true_start == pred_start and true_end == pred_end:
                confusion_matrix[true_type]['TP'] += 1
                matched_pred_indices.add(idx)
                match_found = True
                break
        
        if not match_found:
            confusion_matrix[true_type]['FN'] += 1


    # Any unmatched prediction is a false positive
    for idx, pred_ent in enumerate(y_pred):
        if idx not in matched_pred_indices:
            pred_type = pred_ent[0]
            confusion_matrix[pred_type]['FP'] += 1

    return confusion_matrix

entities_true = Entities(entity_outputs['entity_outputs']['y_true'], scheme, False)
entities_pred = Entities(entity_outputs['entity_outputs']['y_pred'], scheme, False)
true_entity_type = flatten_strict_entities(entities_true)
pred_entity_type = flatten_strict_entities(entities_pred)
# Example usage
# conf_matrix = calculate_confusion_matrix([e.to_tuple()[1:] for sen in entities_true.entities for e in sen], [e.to_tuple()[1:] for sen in entities_pred.entities for e in sen])




In [ ]:
conf_matrix = calculate_confusion_matrix(entity_y_true, entity_y_pred)
print(conf_matrix)

In [ ]:
(627+151+751+338) / ((627+92)+(151+154)+(751+49)+(338+121))

In [ ]:
1867 / 2201

In [ ]:
conf_matrix = calculate_confusion_matrix(true_entity_type, pred_entity_type)
print(conf_matrix)

In [ ]:
total_metrics = {}
for metric in ['TP', 'FP', 'FN']:
    total_metrics[metric] = sum(details[metric] for details in conf_matrix.values())


In [ ]:
total_metrics

In [ ]:
1976 / (1976+189)

In [ ]:
entity_false_positives

In [ ]:

from collections import defaultdict, Counter

def compute_false_negatives(y_true, y_pred):
    fn_counts = defaultdict(Counter)
    true_indexed = {(t[1], t[2]): t[0] for t in y_true}  # Index true entities by boundaries
    pred_indexed = {(p[1], p[2]): p[0] for p in y_pred}  # Index predicted entities by boundaries

    # Iterate through true entities to find false negatives
    for (t_start, t_end), t_type in true_indexed.items():
        if (t_start, t_end) not in pred_indexed or pred_indexed[(t_start, t_end)] != t_type:
            # No matching prediction or type mismatch at the same position
            matched_type = pred_indexed.get((t_start, t_end), 'Boundary')
            fn_counts[t_type][matched_type] += 1

    return fn_counts

def compute_false_positives(y_true, y_pred):
    fp_counts = defaultdict(Counter)
    true_indexed = {(t[1], t[2]): t[0] for t in y_true}  # Index true entities by boundaries
    pred_indexed = {(p[1], p[2]): p[0] for p in y_pred}  # Index predicted entities by boundaries

    # Iterate through predicted entities to find false positives
    for (p_start, p_end), p_type in pred_indexed.items():
        if (p_start, p_end) not in true_indexed or true_indexed[(p_start, p_end)] != p_type:
            # No matching true entity or type mismatch at the same position
            matched_type = true_indexed.get((p_start, p_end), 'Boundary')
            fp_counts[p_type][matched_type] += 1

    return fp_counts

# Example usage
fn_errors = compute_false_negatives(entity_y_true, entity_y_pred)
fp_errors = compute_false_positives(entity_y_true, entity_y_pred)

print("False Negatives:", dict(fn_errors))
print("False Positives:", dict(fp_errors))
1

In [ ]:
import plotly.express as px
import pandas as pd

# Your original data
data = conf_matrix

# Prepare lists for DataFrame construction
actual = []
predicted = []
counts = []

for (act, pred), count in data.items():
    actual.append(act)
    predicted.append('None' if pred is None else pred)  # Replace None with 'None' for better visualization
    counts.append(count)

# Create DataFrame
df = pd.DataFrame({'Actual': actual, 'Predicted': predicted, 'Count': counts})

# Pivot to format suitable for heatmap
pivot_table = df.pivot(index='Actual', columns='Predicted', values='Count').fillna(0)

# Generate heatmap
fig = px.imshow(pivot_table,
                labels=dict(x="Predicted Entity Type", y="Actual Entity Type", color="Count"),
                x=pivot_table.columns,
                y=pivot_table.index,
                text_auto=True,
                aspect="auto")

fig.update_layout(
    title="Entity Recognition Confusion Matrix",
    xaxis_title="Predicted Entity Type",
    yaxis_title="Actual Entity Type"
)

fig.show()


In [ ]:
errors

In [ ]:
import plotly.express as px
import pandas as pd

# Your original data
data = conf_matrix1

# Prepare lists for DataFrame construction
actual = []
predicted = []
counts = []

for (act, pred), count in data.items():
    actual.append(act)
    predicted.append('None' if pred is None else pred)  # Replace None with 'None' for better visualization
    counts.append(count)

# Create DataFrame
df = pd.DataFrame({'Actual': actual, 'Predicted': predicted, 'Count': counts})

# Pivot to format suitable for heatmap
pivot_table = df.pivot(index='Actual', columns='Predicted', values='Count').fillna(0)

# Generate heatmap
fig = px.imshow(pivot_table,
                labels=dict(x="Predicted Entity Type", y="Actual Entity Type", color="Count"),
                x=pivot_table.columns,
                y=pivot_table.index,
                text_auto=True,
                aspect="auto")

fig.update_layout(
    title="Entity Recognition Confusion Matrix",
    xaxis_title="Predicted Entity Type",
    yaxis_title="Actual Entity Type"
)

fig.show()


# Debugging

In [ ]:
ENTITY = 'LOC'
entity_false_negatives = {ENTITY: Counter()}
false_negatives = set([e for e in entity_y_true if e[0] == 'LOC']) - set([e for e in entity_y_pred if e[0] == 'LOC'])
for e in false_negatives:
    t_type, t_start, t_end = e
    for pred_ent in entity_y_pred:
        p_type, p_start, p_end = pred_ent
        if t_start == p_start and t_start == p_end:
            if p_type == 'LOC':
                print(pred_ent)
            entity_false_negatives[t_type][p_type]+=1
            

ENTITY = 'LOC'
entity_false_positives = {ENTITY: Counter()}
false_positive = set([e for e in entity_y_pred if e[0] == ENTITY]) - set([e for e in entity_y_true if e[0] == ENTITY]) 
for e in false_positive:
    p_type, p_start, p_end = e
    for true_ent in entity_y_true:
        t_type, t_start, t_end = true_ent
        if t_start == p_start and t_end == p_end:
            # if p_type == 'ORG':
            #     # if t_type == 'ORG':
            #         print(true_ent)
            if p_type == t_type:
                entity_false_positives[p_type][t_type]+=1

In [ ]:
entity_false_positives

In [ ]:
id = 8786
for entity in entity_y_true:
    t, s, e = entity
    if s == id:
        print(entity)
for entity in entity_y_pred:
    t, s, e = entity
    if s == id or e == id+1:
        print(entity)

In [ ]:
for entity in false_negatives:
    t, s, e = entity
    # if t == 'LOC':
    #     print(entity)
    if s == 8786:
        print(entity)

In [ ]:
for entity in false_positive:
    t, s, e = entity
   
    if s == 16466 or e == 16467:
        print(entity)

In [ ]:
for entity in entity_y_true:
    t, s, e = entity
    if s == 16963:
        print(entity)

In [ ]:
for entity in entity_y_pred:
    t, s, e = entity
    if s == 16963:
        print(entity)

In [ ]:
analysis_data.iloc[8780:8790]